In [150]:
import pandas as pd

# JJ Redick stats
df_jj = pd.read_csv("jj_career.csv")
df_jj.to_json(path_or_buf="jj.json", orient="records")

In [135]:
df_1950_to_2017_totals = pd.read_csv("Seasons_Stats.csv")

In [136]:
df_17_18_avgs = pd.read_csv("nba_2017-2018.csv")
df_18_19_avgs = pd.read_csv("nba_2018-2019.csv")

In [137]:
df_17_18_avgs["Year"] = 2018
df_18_19_avgs["Year"] = 2019
df_17_to_19 = pd.concat([df_17_18_avgs, df_18_19_avgs])

In [138]:
df_17_to_19["Player"] = df_17_to_19["Player"].str.split("\\").str[0]

In [139]:
# change total pts col to AVG
df_1950_to_2017_totals['PTS'] = df.apply(lambda row: row.PTS / row.G, axis=1)

In [140]:
all_seasons = pd.concat([df_1950_to_2017_totals, df_17_to_19])

In [141]:
# Filter to include only seasons with more than 10 games and ppg more than 15
all_seasons_filtered = all_seasons[all_seasons["G"] > 10]
all_seasons_filtered = all_seasons_filtered[all_seasons_filtered["PTS"] > 15]

In [142]:
all_seasons_filtered = all_seasons_filtered[["Player", "Age", "Year", "PTS", "Tm"]]

In [145]:
# Drop players' partial seasons (leave the total)
for temp_tuple in all_seasons_filtered[all_seasons_filtered.Tm == 'TOT'].itertuples():
    part_seasons = all_seasons_filtered[
        (all_seasons_filtered.Player == temp_tuple.Player) & (all_seasons_filtered.Year == temp_tuple.Year) & (all_seasons_filtered.Tm != 'TOT')
    ]
    if len(part_seasons) > 0:
        all_seasons_filtered = all_seasons_filtered.drop(part_seasons.index)

In [146]:
all_seasons_age_ppg = all_seasons_filtered.groupby('Player').apply(lambda x: x.loc[x.PTS == x.PTS.max(),['Player','Age', "PTS", "Year"]])

In [147]:
all_seasons_age_ppg_year = all_seasons_age_ppg.sort_values(by=["Age"], ascending=False).dropna()

In [153]:
top_15_age_ppg = all_seasons_age_ppg_year.head(15)
# formatting numbers
import numpy as np
top_15_age_ppg["Year"] = top_15_age_ppg["Year"].apply(np.int64)
top_15_age_ppg["Age"] = top_15_age_ppg["Age"].apply(np.int64)
top_15_age_ppg = top_15_age_ppg.round(1)

<ipython-input-153-da99d8ed4d9d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_15_age_ppg["Year"] = top_15_age_ppg["Year"].apply(np.int64)
<ipython-input-153-da99d8ed4d9d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_15_age_ppg["Age"] = top_15_age_ppg["Age"].apply(np.int64)


In [154]:
top_15_age_ppg.to_json(path_or_buf="age_ppg.json", orient="records")